In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import joblib

def read_csv_files(directory):
    data_frames = {}
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename))
            parts = filename.split('_')
            # parts[0] = which data, parts[1] = who, parts[2] = difficulty
            file_type = parts[0]
            participant = parts[-2] if len(parts) > 2 else 'unknown'
            difficulty = parts[-1].split('.')[0] if len(parts) > 1 else 'unknown'
            if participant == 'participant1'and not file_type == 'E4 ACC data':
                key = (participant, difficulty, file_type)
                data_frames[key] = df
                
    return data_frames

def preprocess_data(df):
    idx = df[df['Participant Name'] == 'game start'].index[0]
    df = df.drop(index=df.index[:idx])
    if 'E4 TMP' in df.columns:
        df = df.drop(columns = ['E4 TMP'])
    elif 'E4 BVP' in df.columns:
        df = df.drop(columns = ['E4 BVP'])
    
    return df

# 1초 단위로 나눈 dataframe을 받아서 1초 단위로 달라지는 10초짜리 프레임을 가진 데이터 프레임 만들고 arithmetic feature 추가
def make_value(df):
    column_name = df.columns.tolist()
    column_name.remove('Participant Name')
    column_name.remove('UNIX Time')
    column_name_result = []
    for i in column_name:
        column_name_result.extend([f'{i} 1', f'{i} 2', f'{i} 3', f'{i} 4', f'{i} 5', f'{i} 6', f'{i} 7', f'{i} 8', f'{i} 9', f'{i} 10'])
        column_name_result.extend([f'{i} Mean', f'{i} Min', f'{i} Max', f'{i} Std'])
        
    data_frame = pd.DataFrame(columns = column_name_result)
    
    df = df.drop(columns = ['Participant Name', 'UNIX Time'])
    
    x = 0
    while x + 10 < len(df):
        new_row = {}
        for i, col in enumerate(df.columns):
            arith = []
            for j in range(x, x + 10):
                new_row[f'{col} {j - x + 1}'] = df.iloc[j, i]
                arith.append(df.iloc[j, i])
            new_row[f'{col} Mean'] = np.mean(arith)
            new_row[f'{col} Min'] = np.min(arith)
            new_row[f'{col} Max'] = np.max(arith)
            new_row[f'{col} Std'] = np.std(arith)
        x += 1
        data_frame.loc[len(data_frame)] = new_row
    
    return data_frame

# 1초 단위로 나눈 dataframe 만들기
def process_data(df):
    time_window = 1
    df = preprocess_data(df)
    
    data_frame = pd.DataFrame(columns = df.columns.tolist())
    
    idx_before = df[df['Participant Name'] == 'game start'].index[0] + 1
    
    while idx_before + 1 < len(df):
        time = df.loc[idx_before + 1, 'UNIX Time']
        idx_range = df[(df['UNIX Time'] >= time) & (df['UNIX Time'] < time + time_window)].index
        if len(idx_range) == 0:
            break
    
        value = df.loc[idx_before + 1 : idx_range[-1]]
        value = value.median().to_dict()
        
        idx_before = idx_range[-1]

        data_frame.loc[len(data_frame)] = value
    
    result = make_value(data_frame)
    return result
    
if __name__ == "__main__":
    directory = r'C:\Users\USER\Downloads\CSV'  # Use raw string literal
    data_frames = read_csv_files(directory)
    df_data = []
    difficulty_data = []
    
    for (participant, difficulty, file_type), df in data_frames.items():
        df = process_data(df)
        df = df.reset_index(drop=True)
        df_data.append(df)
        difficulty_data.append(difficulty)
    
    # Difficulty = [difficult, easy, moderate]
    
    df_difficult = pd.concat([df_data[0], df_data[3], df_data[6], df_data[9]], axis = 1)
    df_easy = pd.concat([df_data[1], df_data[4], df_data[7], df_data[10]], axis = 1)
    df_moderate = pd.concat([df_data[2], df_data[5], df_data[8], df_data[11]], axis = 1)
    
    df_easy['Difficulty'] = 'easy'
    df_moderate['Difficulty'] = 'moderate'
    df_difficult['Difficulty'] = 'difficult'
    
    df_all = pd.concat([df_easy, df_moderate, df_difficult], axis = 0)
    
    model = RandomForestClassifier()
    x = df_all.drop(columns = ['Difficulty'])
    x.fillna(x.mean(), inplace=True)
    y = df_all[['Difficulty']]
    
    x_list = [x.iloc[i].tolist() for i in range(len(x))]
    y_list = [y.iloc[i].tolist() for i in range(len(y))]
    
    x_train = np.array(x_list)
    y_train = np.array(y_list)
    
    model.fit(x_train, y_train)
    
    result = model.predict(np.array(x_list[0]).reshape(1, -1))
    
    print(result)
    
    joblib_file = "random_forest_model.joblib"
    joblib.dump(model, joblib_file)
    print(f"Model saved as {joblib_file}")

c:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


['easy']
Model saved as random_forest_model.joblib


In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings

# csv 파일 읽기기
def read_csv_files(directory):
    data_frames = {}
    print("\nReading CSV files:")
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            print(f"  Reading: {filename}")
            df = pd.read_csv(os.path.join(directory, filename))
            parts = filename.split('_')
            file_type = parts[0]
            participant = parts[-2] if len(parts) > 2 else 'unknown'
            difficulty = parts[-1].split('.')[0] if len(parts) > 1 else 'unknown'
            if not file_type == 'E4 ACC data':
                key = (participant, difficulty, file_type)
                data_frames[key] = df
            else:
                print(f"    Skipped: {filename} (ACC data)")
    print(f"Total files read: {len(data_frames)}")
    return data_frames

# UNIX Time 자르기
def process_data(df):
    if 'started' in df['game started'].values:
        idx = df[df['game started'] == 'started'].index[0]
        df = df.iloc[idx:]
    else:
        print("Warning: 'game start' marker not found. Using all data.")
    
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].interpolate(method='linear', limit_direction='both')
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
    
    if 'UNIX Time' not in df.columns or df['UNIX Time'].isnull().all():
        print("Error: 'UNIX Time' column not found or all values are null")
        return pd.DataFrame()
    df_numeric = df.select_dtypes(include=[np.number]).drop(columns=['UNIX Time'])
    
    return df_numeric

# window size 크기로 잘라서 sequence로 만들기
def make_sequences(df, window_size):
    sequences = []
    for i in range(0, len(df) - window_size + 1):
        sequences.extend(df.iloc[i:i+window_size].values)
    return np.array(sequences)

# 훈련 데이터셋 X_train, Y_train 만들기
def make_dataset(data_frames, window_size):
    X_data = []
    y_data = []
    all_columns = set()
    
    for (participant, difficulty, file_type), df in data_frames.items():
        processed_df = process_data(df)
        all_columns.update(processed_df.columns)
    
    all_columns = sorted(list(all_columns))  # Limit to expected number of features
    
    for (participant, difficulty, file_type), df in data_frames.items():
        processed_df = process_data(df)
        
        for col in all_columns:
            if col not in processed_df.columns:
                processed_df[col] = 0
        
        processed_df = processed_df[all_columns]
            
        sequences = make_sequences(processed_df, window_size)
        if len(sequences) > 0:
            X_data.append(sequences)
            y_data.extend([difficulty] * len(sequences))
            
    label_encoder = LabelEncoder()
    y_data = label_encoder.fit_transform(y_data)
    
    X_train = np.concatenate(X_data)
    Y_train = np.array(y_data)
    return X_train, Y_train

# 5-fold cross validation과 하이퍼 파라미터 조정을 통해서 학습 및 정확도 출력
def train_model(X_train, Y_train):
    print("Training started from now")
    
    # Hyperparameter tuning using GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['sqrt', 'log2']
    }
    
    # 5-Fold Cross Validation with GridSearchCV
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, test_idx in kf.split(X_train, Y_train):
        X_train_fold, X_test_fold = X_train[train_idx], X_train[test_idx]
        Y_train_fold, Y_test_fold = Y_train[train_idx], Y_train[test_idx]
        
        rf = RandomForestClassifier(random_state=42)
        rf.fit(X_train_fold, Y_train_fold)
        
        y_pred = rf.predict(X_test_fold)
        accuracy = accuracy_score(Y_test_fold, y_pred)
        accuracies.append(accuracy)
    
    # Output the results
    print(f'Average Accuracy: {np.mean(accuracies):.4f}')

def main():
    directory = r'C:\Users\USER\Downloads\pilotdata'
    data_frames = read_csv_files(directory)
    
    window_size = 10
    
    X_train, Y_train = make_dataset(data_frames, window_size)
    X_train = np.nan_to_num(X_train, nan=0.0)
    
    if X_train is None or Y_train is None:
        print("Failed to create dataset.")
        return
    
    train_model(X_train, Y_train)
    
if __name__ == "__main__":
    main()


Reading CSV files:
  Reading: differential_cy_easier.csv
  Reading: differential_cy_easy.csv
  Reading: differential_cy_hard.csv
  Reading: differential_cy_harder.csv
  Reading: differential_cy_moderate.csv
  Reading: differential_dh_easier.csv
  Reading: differential_dh_easy.csv
  Reading: differential_dh_hard.csv
  Reading: differential_dh_harder.csv
  Reading: differential_dh_moderate.csv
  Reading: differential_gb_easier.csv
  Reading: differential_gb_easy.csv
  Reading: differential_gb_hard.csv
  Reading: differential_gb_harder.csv
  Reading: differential_gb_moderate.csv
  Reading: differential_jj_easier.csv
  Reading: differential_jj_easy.csv
  Reading: differential_jj_hard.csv
  Reading: differential_jj_harder.csv
  Reading: differential_jj_moderate.csv
  Reading: differential_jw_easier.csv
  Reading: differential_jw_easy.csv
  Reading: differential_jw_hard.csv
  Reading: differential_jw_harder.csv
  Reading: differential_jw_moderate.csv
  Reading: differential_kb_easier.csv
  